# Import library and split data

In [8]:
import pandas as pd

In [9]:
df = pd.read_csv('../../Data/BPD_related1.csv')
df

,Send Rates,Block Size,Avg Latency,Throughput
0,10.0,10,0.33,10.0
1,10.0,20,0.85,10.0
2,10.0,30,1.19,10.0
3,10.0,40,1.22,9.8
4,10.0,50,1.18,10.0
...,...,...,...,...
395,200.5,160,1.35,160.8
396,187.5,170,2.17,158.4
397,198.9,180,1.31,163.5
398,197.0,190,1.65,163.0


In [10]:
xy = df.values
xy

array([[ 10.  ,  10.  ,   0.33,  10.  ],
       [ 10.  ,  20.  ,   0.85,  10.  ],
       [ 10.  ,  30.  ,   1.19,  10.  ],
       ...,
       [198.9 , 180.  ,   1.31, 163.5 ],
       [197.  , 190.  ,   1.65, 163.  ],
       [198.3 , 200.  ,   1.63, 161.6 ]])

In [11]:
import torch
import numpy as np
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [12]:
# set random seed
seed = 42
torch.manual_seed(seed)

In [13]:
# Read raw data and divide training set and test set
raw_data = xy
X = raw_data[:, :2]
Y1 = raw_data[:, -2:-1]
Y2 = raw_data[:, -1:]

In [14]:
X

array([[ 10. ,  10. ],
       [ 10. ,  20. ],
       [ 10. ,  30. ],
       [ 10. ,  40. ],
       [ 10. ,  50. ],
       [ 10. ,  60. ],
       [ 10. ,  70. ],
       [ 10. ,  80. ],
       [ 10. ,  90. ],
       [ 10.1, 100. ],
       [ 10. , 110. ],
       [ 10. , 120. ],
       [ 10. , 130. ],
       [ 10. , 140. ],
       [ 10. , 150. ],
       [ 10. , 160. ],
       [ 10. , 170. ],
       [ 10. , 180. ],
       [ 10. , 190. ],
       [ 10. , 200. ],
       [ 20.1,  10. ],
       [ 20.1,  20. ],
       [ 20.1,  30. ],
       [ 20.1,  40. ],
       [ 20.1,  50. ],
       [ 20.1,  60. ],
       [ 20.1,  70. ],
       [ 20.1,  80. ],
       [ 20.1,  90. ],
       [ 20.1, 100. ],
       [ 20.1, 110. ],
       [ 20.1, 120. ],
       [ 20.1, 130. ],
       [ 20.1, 140. ],
       [ 20.1, 150. ],
       [ 20.1, 160. ],
       [ 20.1, 170. ],
       [ 20.1, 180. ],
       [ 20.1, 190. ],
       [ 20.1, 200. ],
       [ 30.1,  10. ],
       [ 30.1,  20. ],
       [ 30.1,  30. ],
       [ 30

In [15]:
Y1

array([[0.33],
       [0.85],
       [1.19],
       [1.22],
       [1.18],
       [1.19],
       [1.17],
       [1.13],
       [1.17],
       [1.17],
       [1.18],
       [1.18],
       [1.15],
       [1.18],
       [1.17],
       [1.17],
       [1.14],
       [1.16],
       [1.13],
       [1.12],
       [0.19],
       [0.46],
       [0.74],
       [0.99],
       [1.2 ],
       [1.16],
       [1.16],
       [1.16],
       [1.15],
       [1.18],
       [1.18],
       [1.16],
       [1.18],
       [1.16],
       [1.14],
       [1.18],
       [1.16],
       [1.17],
       [1.17],
       [1.16],
       [0.13],
       [0.31],
       [0.51],
       [0.71],
       [0.85],
       [1.04],
       [1.16],
       [1.16],
       [1.15],
       [1.14],
       [1.16],
       [1.14],
       [1.16],
       [1.16],
       [1.16],
       [1.14],
       [1.16],
       [1.17],
       [1.16],
       [1.15],
       [0.11],
       [0.24],
       [0.41],
       [0.67],
       [0.65],
       [0.81],
       [0.

In [16]:
Y2

array([[ 10. ],
       [ 10. ],
       [ 10. ],
       [  9.8],
       [ 10. ],
       [ 10. ],
       [  9.9],
       [  9.9],
       [ 10. ],
       [ 10. ],
       [ 10. ],
       [ 10. ],
       [  9.9],
       [  9.9],
       [ 10. ],
       [  9.8],
       [  9.8],
       [ 10. ],
       [ 10. ],
       [ 10. ],
       [ 20.1],
       [ 20.1],
       [ 19.4],
       [ 20. ],
       [ 19.9],
       [ 20. ],
       [ 19.3],
       [ 19.9],
       [ 19.6],
       [ 19.9],
       [ 19.8],
       [ 20.1],
       [ 19.9],
       [ 19.3],
       [ 19.4],
       [ 19.8],
       [ 20. ],
       [ 19.4],
       [ 19.9],
       [ 19.4],
       [ 30.1],
       [ 30.1],
       [ 28.5],
       [ 30.1],
       [ 30.1],
       [ 29.3],
       [ 28.8],
       [ 28.6],
       [ 28.6],
       [ 28.7],
       [ 28.5],
       [ 28.6],
       [ 28.5],
       [ 28.6],
       [ 28.4],
       [ 28.5],
       [ 28.6],
       [ 28.6],
       [ 28.4],
       [ 28.8],
       [ 40.1],
       [ 40.1],
       [

# MLP1(send rates, block size) = latency

In [17]:
# Split training set and test set
Xtrain1, Xtest1, Ytrain1, Ytest1 = train_test_split(X, Y1, test_size=0.2, random_state=1)

In [18]:
# test set
Xtest1

array([[197. , 190. ],
       [ 70.2,  60. ],
       [170.2,  90. ],
       [170.3, 200. ],
       [ 90.5, 130. ],
       [179. ,  30. ],
       [100.4, 180. ],
       [150.1, 120. ],
       [ 20.1, 100. ],
       [150.2,  50. ],
       [ 90.4, 150. ],
       [190.5, 130. ],
       [100.3,  90. ],
       [170.5,  50. ],
       [167.2,  20. ],
       [120.5,  80. ],
       [189.3, 120. ],
       [ 10. ,  60. ],
       [ 40.2, 190. ],
       [120.4,  40. ],
       [ 70.3,  30. ],
       [130.4,  30. ],
       [191.5,  30. ],
       [110.3, 150. ],
       [ 10. , 180. ],
       [ 50.3, 130. ],
       [190.3,  70. ],
       [110.2,  20. ],
       [187.7,  20. ],
       [110.6,  80. ],
       [ 50.2,  20. ],
       [ 10. ,  50. ],
       [ 90.4,  60. ],
       [140.6, 160. ],
       [ 10. ,  70. ],
       [ 50.2,  10. ],
       [ 30.1, 190. ],
       [ 60.3,  30. ],
       [198.9, 180. ],
       [ 70.3, 200. ],
       [140.4,  90. ],
       [110.5,  50. ],
       [ 90.4, 120. ],
       [ 40

In [14]:
Ytest1

array([[2.99],
       [1.18],
       [0.33],
       [1.16],
       [1.15],
       [1.55],
       [0.9 ],
       [0.81],
       [1.2 ],
       [1.22],
       [1.23],
       [1.28],
       [0.92],
       [1.18],
       [0.91],
       [0.26],
       [1.18],
       [1.26],
       [1.17],
       [0.87],
       [1.18],
       [0.92],
       [0.94],
       [1.24],
       [0.88],
       [1.19],
       [2.32],
       [1.01],
       [1.66],
       [0.76],
       [0.77],
       [1.34],
       [0.89],
       [1.19],
       [0.65],
       [0.83],
       [1.13],
       [2.57],
       [0.92],
       [1.2 ],
       [0.97],
       [0.89],
       [0.88],
       [1.16],
       [2.29],
       [0.47],
       [1.15],
       [0.93],
       [0.85],
       [0.8 ],
       [1.19],
       [2.56],
       [1.18],
       [1.73],
       [1.23],
       [1.45],
       [2.1 ],
       [1.27],
       [1.52],
       [0.85],
       [1.17],
       [0.95],
       [0.86],
       [0.25],
       [0.84],
       [0.9 ],
       [1.

In [15]:
# property scaling
min_max_scaler1 = preprocessing.MinMaxScaler()

In [16]:
# Scaling training set data
Xtrain1_minmax = min_max_scaler1.fit_transform(Xtrain1)
Xtrain1_minmax

array([[0.8427673 , 0.51898734],
       [0.39412998, 0.92405063],
       [0.52568134, 0.46835443],
       ...,
       [0.34224319, 0.70886076],
       [0.05293501, 0.94936709],
       [0.34224319, 0.26582278]])

In [17]:
# Apply the same scaling to the test set data
Xtest1_minmax = min_max_scaler1.transform(Xtest1)
Xtest1_minmax

array([[0.94444444, 0.89873418],
       [0.02672956, 0.37974684],
       [0.34224319, 0.03797468],
       ...,
       [0.5       , 0.5443038 ],
       [0.47222222, 0.08860759],
       [0.52672956, 0.01265823]])

In [18]:
Xtest1_tensor = torch.from_numpy(Xtest1_minmax).type(torch.float32)
Ytest1_tensor = torch.from_numpy(Ytest1).type(torch.float32)

In [19]:
Xtest1_tensor

tensor([[0.9444, 0.8987],
        [0.0267, 0.3797],
        [0.3422, 0.0380],
        ...,
        [0.5000, 0.5443],
        [0.4722, 0.0886],
        [0.5267, 0.0127]])

In [20]:
# batch the training dataset
# prepare dataset
class BlockChainDataset(Dataset):
    def __init__(self, data, label):
        self.len = data.shape[0]
        self.x_data = torch.from_numpy(data).type(torch.float32)
        self.y_data = torch.from_numpy(label).type(torch.float32)
        
        
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]
    
    def __len__(self):
        return self.len

In [21]:
train_dataset = BlockChainDataset(Xtrain1_minmax, Ytrain1)
train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=False)

In [22]:
import torch.nn as nn
import torch.optim as optim

In [23]:
# design model using class
class Lat(torch.nn.Module):
    def __init__(self):
        super(Lat, self).__init__()
        self.fc1 = nn.Linear(2, 64)
        self.fc2 = nn.Linear(64, 8)
        self.relu = nn.ReLU()
        self.fc3 = nn.Linear(8, 1)
        self.dropout = nn.Dropout(p=0.5)
        
    def forward(self, x):
        out = self.fc1(x)
        out = self.dropout(out)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.dropout(out)
        out = self.relu(out)
        out = self.fc3(out)
        return out
    
    
model = Lat()

In [69]:
# net = nn.Sequential(
#     nn.Linear(2, 64), nn.BatchNorm1d(64), nn.Dropout(p=0.5), nn.ReLU(),
#     nn.Linear(64, 64), nn.BatchNorm1d(64), nn.Dropout(p=0.5), nn.ReLU(),
#     nn.Linear(64, 1))


# model = net

In [24]:
# construct loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [25]:
# traning cycle forward, backward, update
def train(epoch):
    train_loss = 0.0
    for batch_idx, (inputs, labels) in enumerate(train_loader, 0):
        y_pred = model(inputs)
        loss = criterion(y_pred, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * labels.shape[0]
    if (epoch + 1) % 10 == 0:
        print('epoch:', epoch + 1, 'train_loss:', train_loss / len(Xtrain1))
        

def test():
    with torch.no_grad():
        y_pred = model(Xtest1_tensor)
        loss = criterion(y_pred, Ytest1_tensor)
        print('test_loss:', loss)

In [44]:
if __name__ == '__main__':
    for epoch in range(5000):
        train(epoch)
        if (epoch + 1) % 10 == 0:
            model.eval()
            test()

epoch: 10 train_loss: 0.052233589068055156
test_loss: tensor(0.0300)
epoch: 20 train_loss: 0.05219027679413557
test_loss: tensor(0.0300)
epoch: 30 train_loss: 0.0521331487223506
test_loss: tensor(0.0300)
epoch: 40 train_loss: 0.05210463274270296
test_loss: tensor(0.0300)
epoch: 50 train_loss: 0.052052390016615394
test_loss: tensor(0.0300)
epoch: 60 train_loss: 0.05200352035462856
test_loss: tensor(0.0300)
epoch: 70 train_loss: 0.05197533424943686
test_loss: tensor(0.0299)
epoch: 80 train_loss: 0.05186860896646976
test_loss: tensor(0.0300)
epoch: 90 train_loss: 0.05173366386443377
test_loss: tensor(0.0296)
epoch: 100 train_loss: 0.05158418510109186
test_loss: tensor(0.0295)
epoch: 110 train_loss: 0.051470202207565305
test_loss: tensor(0.0293)
epoch: 120 train_loss: 0.051355143822729585
test_loss: tensor(0.0294)
epoch: 130 train_loss: 0.05126049015671015
test_loss: tensor(0.0293)
epoch: 140 train_loss: 0.051125096157193184
test_loss: tensor(0.0293)
epoch: 150 train_loss: 0.05101311337202

epoch: 1210 train_loss: 0.04490919839590788
test_loss: tensor(0.0269)
epoch: 1220 train_loss: 0.044886069186031816
test_loss: tensor(0.0267)
epoch: 1230 train_loss: 0.044877889566123486
test_loss: tensor(0.0267)
epoch: 1240 train_loss: 0.04477553674951196
test_loss: tensor(0.0268)
epoch: 1250 train_loss: 0.044747271295636895
test_loss: tensor(0.0268)
epoch: 1260 train_loss: 0.044730660133063795
test_loss: tensor(0.0267)
epoch: 1270 train_loss: 0.044691288471221925
test_loss: tensor(0.0267)
epoch: 1280 train_loss: 0.04461910193786025
test_loss: tensor(0.0269)
epoch: 1290 train_loss: 0.044587109610438345
test_loss: tensor(0.0267)
epoch: 1300 train_loss: 0.044579712115228175
test_loss: tensor(0.0267)
epoch: 1310 train_loss: 0.04455103762447834
test_loss: tensor(0.0267)
epoch: 1320 train_loss: 0.04446751531213522
test_loss: tensor(0.0268)
epoch: 1330 train_loss: 0.04445881461724639
test_loss: tensor(0.0267)
epoch: 1340 train_loss: 0.044440907519310714
test_loss: tensor(0.0267)
epoch: 1350 

epoch: 2390 train_loss: 0.041733233630657195
test_loss: tensor(0.0259)
epoch: 2400 train_loss: 0.04171702554449439
test_loss: tensor(0.0260)
epoch: 2410 train_loss: 0.041728274524211885
test_loss: tensor(0.0260)
epoch: 2420 train_loss: 0.041691673919558525
test_loss: tensor(0.0259)
epoch: 2430 train_loss: 0.041673250030726194
test_loss: tensor(0.0260)
epoch: 2440 train_loss: 0.041664407774806025
test_loss: tensor(0.0260)
epoch: 2450 train_loss: 0.04165559923276305
test_loss: tensor(0.0260)
epoch: 2460 train_loss: 0.0416522653773427
test_loss: tensor(0.0260)
epoch: 2470 train_loss: 0.04160564299672842
test_loss: tensor(0.0261)
epoch: 2480 train_loss: 0.04157986268401146
test_loss: tensor(0.0261)
epoch: 2490 train_loss: 0.04157840646803379
test_loss: tensor(0.0260)
epoch: 2500 train_loss: 0.041568286903202536
test_loss: tensor(0.0261)
epoch: 2510 train_loss: 0.041552466060966256
test_loss: tensor(0.0261)
epoch: 2520 train_loss: 0.04153741719201207
test_loss: tensor(0.0261)
epoch: 2530 tr

epoch: 3560 train_loss: 0.04075597831979394
test_loss: tensor(0.0275)
epoch: 3570 train_loss: 0.04074216037988663
test_loss: tensor(0.0274)
epoch: 3580 train_loss: 0.04075707821175456
test_loss: tensor(0.0274)
epoch: 3590 train_loss: 0.04073150521144271
test_loss: tensor(0.0276)
epoch: 3600 train_loss: 0.040778305102139714
test_loss: tensor(0.0276)
epoch: 3610 train_loss: 0.04074556352570653
test_loss: tensor(0.0275)
epoch: 3620 train_loss: 0.04074637740850449
test_loss: tensor(0.0274)
epoch: 3630 train_loss: 0.040740808844566344
test_loss: tensor(0.0275)
epoch: 3640 train_loss: 0.040728802978992465
test_loss: tensor(0.0274)
epoch: 3650 train_loss: 0.04071098323911428
test_loss: tensor(0.0274)
epoch: 3660 train_loss: 0.04070665091276169
test_loss: tensor(0.0276)
epoch: 3670 train_loss: 0.040751616656780246
test_loss: tensor(0.0276)
epoch: 3680 train_loss: 0.04072785498574376
test_loss: tensor(0.0276)
epoch: 3690 train_loss: 0.04072588579729199
test_loss: tensor(0.0274)
epoch: 3700 trai

epoch: 4730 train_loss: 0.04040937945246696
test_loss: tensor(0.0282)
epoch: 4740 train_loss: 0.040391347277909516
test_loss: tensor(0.0282)
epoch: 4750 train_loss: 0.040411506127566096
test_loss: tensor(0.0281)
epoch: 4760 train_loss: 0.040400182269513606
test_loss: tensor(0.0279)
epoch: 4770 train_loss: 0.040404736343771216
test_loss: tensor(0.0282)
epoch: 4780 train_loss: 0.04041217658668757
test_loss: tensor(0.0283)
epoch: 4790 train_loss: 0.040405870601534846
test_loss: tensor(0.0282)
epoch: 4800 train_loss: 0.04039819771423936
test_loss: tensor(0.0281)
epoch: 4810 train_loss: 0.04038372738286853
test_loss: tensor(0.0279)
epoch: 4820 train_loss: 0.04038691986352205
test_loss: tensor(0.0283)
epoch: 4830 train_loss: 0.04036543676629663
test_loss: tensor(0.0281)
epoch: 4840 train_loss: 0.040365563612431286
test_loss: tensor(0.0281)
epoch: 4850 train_loss: 0.040371925383806226
test_loss: tensor(0.0282)
epoch: 4860 train_loss: 0.04037431087344885
test_loss: tensor(0.0281)
epoch: 4870 t

In [45]:
Xtest1

array([[197. , 190. ],
       [ 70.2,  60. ],
       [170.2,  90. ],
       [170.3, 200. ],
       [ 90.5, 130. ],
       [179. ,  30. ],
       [100.4, 180. ],
       [150.1, 120. ],
       [ 20.1, 100. ],
       [150.2,  50. ],
       [ 90.4, 150. ],
       [190.5, 130. ],
       [100.3,  90. ],
       [170.5,  50. ],
       [167.2,  20. ],
       [120.5,  80. ],
       [189.3, 120. ],
       [ 10. ,  60. ],
       [ 40.2, 190. ],
       [120.4,  40. ],
       [ 70.3,  30. ],
       [130.4,  30. ],
       [191.5,  30. ],
       [110.3, 150. ],
       [ 10. , 180. ],
       [ 50.3, 130. ],
       [190.3,  70. ],
       [110.2,  20. ],
       [187.7,  20. ],
       [110.6,  80. ],
       [ 50.2,  20. ],
       [ 10. ,  50. ],
       [ 90.4,  60. ],
       [140.6, 160. ],
       [ 10. ,  70. ],
       [ 50.2,  10. ],
       [ 30.1, 190. ],
       [ 60.3,  30. ],
       [198.9, 180. ],
       [ 70.3, 200. ],
       [140.4,  90. ],
       [110.5,  50. ],
       [ 90.4, 120. ],
       [ 40

In [46]:
Xtest1[:, 1].reshape(-1)

array([190.,  60.,  90., 200., 130.,  30., 180., 120., 100.,  50., 150.,
       130.,  90.,  50.,  20.,  80., 120.,  60., 190.,  40.,  30.,  30.,
        30., 150., 180., 130.,  70.,  20.,  20.,  80.,  20.,  50.,  60.,
       160.,  70.,  10., 190.,  30., 180., 200.,  90.,  50., 120.,  30.,
        60.,  60., 100.,  80., 150., 190.,  60.,  60., 100., 190., 170.,
        70., 130., 180., 190.,  10.,  80.,  80., 150.,  70., 150.,  30.,
        70., 110., 110., 130., 190., 160., 110.,  80., 120.,  80.,  20.,
        10., 130.,  50.])

In [47]:
with torch.no_grad():
    y_pred = model(Xtest1_tensor)
y = pd.concat([pd.Series(Xtest1[:, 0].reshape(-1), name='send rates'), pd.Series(Xtest1[:, 1].reshape(-1), name='block size'), 
               pd.Series(Ytest1_tensor.numpy().reshape(-1), name='latency_true'), pd.Series(y_pred.numpy().reshape(-1), name='latency_pred')], 
              axis=1)
y.to_csv('./latency_true_pred_related1.csv')

In [48]:
# Compute test set MAE RMSE MAPE
import pandas as pd

data = pd.read_csv('./latency_true_pred_related1.csv')
data.drop('Unnamed: 0', axis=1, inplace=True)
data.head()

,send rates,block size,latency_true,latency_pred
0,197.0,190.0,1.65,1.497848
1,70.2,60.0,0.56,0.553954
2,170.2,90.0,1.21,0.793090
3,170.3,200.0,0.92,0.913903
4,90.5,130.0,0.90,0.813841


In [49]:
y = data['latency_true'].values
y

array([1.65, 0.56, 1.21, 0.92, 0.9 , 1.68, 0.76, 0.81, 1.18, 0.81, 0.83,
       1.69, 0.6 , 1.65, 2.37, 0.57, 1.43, 1.19, 1.16, 0.39, 0.29, 0.59,
       1.85, 0.85, 1.16, 1.23, 1.01, 0.35, 1.91, 0.58, 0.21, 1.18, 0.47,
       0.79, 1.17, 0.09, 1.16, 0.29, 1.31, 1.02, 0.55, 0.5 , 0.95, 0.41,
       1.11, 0.68, 1.57, 0.6 , 0.96, 0.8 , 0.81, 0.44, 0.79, 1.13, 0.9 ,
       0.43, 0.75, 1.19, 0.83, 1.64, 0.79, 1.08, 0.82, 0.44, 1.49, 0.29,
       0.67, 0.76, 0.72, 1.2 , 0.85, 1.19, 1.01, 1.48, 1.18, 0.87, 0.31,
       0.11, 1.08, 0.44])

In [50]:
y_hat = data['latency_pred'].values
y_hat

array([1.4978478 , 0.55395377, 0.7930897 , 0.9139033 , 0.81384116,
       1.3864472 , 0.75785774, 0.7757339 , 1.1435018 , 0.6058958 ,
       0.82730526, 1.4952234 , 0.66688854, 1.3293817 , 1.5366441 ,
       0.52950794, 1.4209621 , 1.3794163 , 1.1534368 , 0.67124176,
       0.30876976, 0.32951614, 1.557362  , 0.7915691 , 1.1337626 ,
       1.1521361 , 0.80856436, 0.45578927, 1.815589  , 0.540561  ,
       0.23321153, 1.4051139 , 0.48424473, 0.82691765, 1.3100386 ,
       0.09630895, 1.1488233 , 0.31633332, 1.565887  , 1.0352813 ,
       0.599356  , 0.4260105 , 0.8152727 , 0.4051596 , 1.1907303 ,
       0.6916644 , 1.3075556 , 0.52727026, 0.9106764 , 0.7943826 ,
       0.81004256, 0.43839845, 0.72446626, 1.1396426 , 0.84982854,
       0.42835286, 0.91967607, 1.1101464 , 0.7870315 , 1.8667512 ,
       0.72078514, 0.9684507 , 0.81682146, 0.45555633, 1.2282456 ,
       0.30846006, 0.68124825, 0.8505971 , 0.7438759 , 0.95582306,
       0.7785057 , 1.155096  , 1.2005758 , 1.0324607 , 1.14309

In [51]:
from sklearn import metrics

In [52]:
MAE = metrics.mean_absolute_error(y, y_hat)
RMSE = metrics.mean_squared_error(y, y_hat) ** 0.5
MAPE = metrics.mean_absolute_percentage_error(y, y_hat)
print('MAE: ', MAE)
print('RMSE: ', RMSE)
print('MAPE: ', MAPE)

MAE:  0.104048277425
RMSE:  0.1684201009354377
MAPE:  0.10662073111580077


# MLP2(send rates, block size) = throughput

In [53]:
# Split training set and test set
Xtrain2, Xtest2, Ytrain2, Ytest2 = train_test_split(X, Y2, test_size=0.2, random_state=1)

In [54]:
Xtest2

array([[197. , 190. ],
       [ 70.2,  60. ],
       [170.2,  90. ],
       [170.3, 200. ],
       [ 90.5, 130. ],
       [179. ,  30. ],
       [100.4, 180. ],
       [150.1, 120. ],
       [ 20.1, 100. ],
       [150.2,  50. ],
       [ 90.4, 150. ],
       [190.5, 130. ],
       [100.3,  90. ],
       [170.5,  50. ],
       [167.2,  20. ],
       [120.5,  80. ],
       [189.3, 120. ],
       [ 10. ,  60. ],
       [ 40.2, 190. ],
       [120.4,  40. ],
       [ 70.3,  30. ],
       [130.4,  30. ],
       [191.5,  30. ],
       [110.3, 150. ],
       [ 10. , 180. ],
       [ 50.3, 130. ],
       [190.3,  70. ],
       [110.2,  20. ],
       [187.7,  20. ],
       [110.6,  80. ],
       [ 50.2,  20. ],
       [ 10. ,  50. ],
       [ 90.4,  60. ],
       [140.6, 160. ],
       [ 10. ,  70. ],
       [ 50.2,  10. ],
       [ 30.1, 190. ],
       [ 60.3,  30. ],
       [198.9, 180. ],
       [ 70.3, 200. ],
       [140.4,  90. ],
       [110.5,  50. ],
       [ 90.4, 120. ],
       [ 40

In [55]:
Ytest2

array([[163. ],
       [ 63.2],
       [128.5],
       [159.6],
       [ 80.6],
       [129.1],
       [ 98.6],
       [117.4],
       [ 19.9],
       [118. ],
       [ 81.3],
       [159.4],
       [ 83.3],
       [125.4],
       [122.6],
       [ 99.1],
       [134.4],
       [ 10. ],
       [ 37.7],
       [119.3],
       [ 61.7],
       [103.5],
       [134.1],
       [ 96.7],
       [ 10. ],
       [ 47.8],
       [134.9],
       [ 90.9],
       [133.5],
       [ 92.3],
       [ 50.1],
       [ 10. ],
       [ 78.3],
       [121.5],
       [  9.9],
       [ 50.1],
       [ 28.4],
       [ 53.9],
       [163.5],
       [ 69.8],
       [109.1],
       [ 92.7],
       [ 78.4],
       [ 37.3],
       [139.4],
       [ 46.9],
       [138.7],
       [123.1],
       [121.6],
       [121.4],
       [ 38.3],
       [ 98.9],
       [103.3],
       [ 10. ],
       [120.7],
       [116. ],
       [121.4],
       [ 59.5],
       [121.9],
       [126.8],
       [ 55.2],
       [ 38.3],
       [

In [56]:
# property scaling
min_max_scaler2 = preprocessing.MinMaxScaler()

In [57]:
# Scaling training set data
Xtrain2_minmax = min_max_scaler2.fit_transform(Xtrain2)
Xtrain2_minmax

array([[2.10802307e-01, 6.84210526e-01],
       [5.29627687e-02, 1.57894737e-01],
       [7.35186156e-01, 1.00000000e+00],
       [0.00000000e+00, 6.84210526e-01],
       [2.10802307e-01, 5.26315789e-01],
       [1.58363922e-01, 5.26315789e-02],
       [9.17671736e-01, 5.26315789e-02],
       [5.25957001e-01, 6.84210526e-01],
       [9.83744101e-01, 5.78947368e-01],
       [3.16727845e-01, 3.68421053e-01],
       [3.68641846e-01, 1.00000000e+00],
       [8.96696382e-01, 7.89473684e-01],
       [7.28893550e-01, 1.57894737e-01],
       [8.40587310e-01, 9.47368421e-01],
       [4.21604615e-01, 5.26315789e-02],
       [7.89722077e-01, 0.00000000e+00],
       [6.84320923e-01, 8.94736842e-01],
       [7.36234924e-01, 4.73684211e-01],
       [6.84845307e-01, 0.00000000e+00],
       [4.21080231e-01, 6.84210526e-01],
       [2.10802307e-01, 1.05263158e-01],
       [9.46512847e-01, 3.68421053e-01],
       [4.21080231e-01, 1.00000000e+00],
       [5.29627687e-02, 3.68421053e-01],
       [2.108023

In [58]:
# Apply the same scaling to the test set data
Xtest2_minmax = min_max_scaler2.transform(Xtest2)
Xtest2_minmax

array([[0.9805978 , 0.94736842],
       [0.31567908, 0.26315789],
       [0.84006293, 0.42105263],
       [0.84058731, 1.        ],
       [0.422129  , 0.63157895],
       [0.8862087 , 0.10526316],
       [0.474043  , 0.89473684],
       [0.73466177, 0.57894737],
       [0.05296277, 0.47368421],
       [0.73518616, 0.21052632],
       [0.42160461, 0.73684211],
       [0.94651285, 0.63157895],
       [0.47351862, 0.42105263],
       [0.84163608, 0.21052632],
       [0.82433141, 0.05263158],
       [0.57944415, 0.36842105],
       [0.94022024, 0.57894737],
       [0.        , 0.26315789],
       [0.15836392, 0.94736842],
       [0.57891977, 0.15789474],
       [0.31620346, 0.10526316],
       [0.63135815, 0.10526316],
       [0.95175669, 0.10526316],
       [0.525957  , 0.73684211],
       [0.        , 0.89473684],
       [0.21132669, 0.63157895],
       [0.94546408, 0.31578947],
       [0.52543262, 0.05263158],
       [0.9318301 , 0.05263158],
       [0.52753015, 0.36842105],
       [0.

In [59]:
Xtest2_tensor = torch.from_numpy(Xtest2_minmax).type(torch.float32)
Ytest2_tensor = torch.from_numpy(Ytest2).type(torch.float32)

In [60]:
train_dataset = BlockChainDataset(Xtrain2_minmax, Ytrain2)
train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=False)

In [61]:
# design model using class
class Thr(torch.nn.Module):
    def __init__(self):
        super(Thr, self).__init__()
        self.fc1 = nn.Linear(2, 64)
        self.fc2 = nn.Linear(64, 8)
        self.relu = nn.ReLU()
        self.fc3 = nn.Linear(8, 1)
        self.dropout = nn.Dropout(p=0.5)
        
    def forward(self, x):
        out = self.fc1(x)
        out = self.dropout(out)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.dropout(out)
        out = self.relu(out)
        out = self.fc3(out)
        return out
    
    
model1 = Thr()

In [62]:
# construct loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model1.parameters(), lr=0.001)

In [ ]:
# traning cycle forward, backward, update
def train1(epoch):
    train_loss = 0.0
    for batch_idx, (inputs, labels) in enumerate(train_loader, 0):
        y_pred = model1(inputs)
        loss = criterion(y_pred, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * labels.shape[0]
    if (epoch + 1) % 10 == 0:
        print('epoch:', epoch + 1, 'train_loss:', train_loss / len(Xtrain2))
        

def test1():
    with torch.no_grad():
        y_pred = model1(Xtest2_tensor)
        loss = criterion(y_pred, Ytest2_tensor)
        print('test_loss:', loss)

In [67]:
if __name__ == '__main__':
    for epoch in range(5000):
        train1(epoch)
        if (epoch + 1) % 10 == 0:
            model1.eval()
            test1()

epoch: 10 train_loss: 40.42617921829223
test_loss: tensor(35.9523)
epoch: 20 train_loss: 40.424303722381595
test_loss: tensor(35.9711)
epoch: 30 train_loss: 40.42428321838379
test_loss: tensor(35.9714)
epoch: 40 train_loss: 40.42399415969849
test_loss: tensor(35.9743)
epoch: 50 train_loss: 40.424316787719725
test_loss: tensor(35.9721)
epoch: 60 train_loss: 40.42429962158203
test_loss: tensor(35.9750)
epoch: 70 train_loss: 40.424763679504395
test_loss: tensor(35.9729)
epoch: 80 train_loss: 40.42577991485596
test_loss: tensor(35.9808)
epoch: 90 train_loss: 40.42421588897705
test_loss: tensor(35.9773)
epoch: 100 train_loss: 40.42426528930664
test_loss: tensor(35.9746)
epoch: 110 train_loss: 40.42396202087402
test_loss: tensor(35.9761)
epoch: 120 train_loss: 40.42429389953613
test_loss: tensor(35.9733)
epoch: 130 train_loss: 40.42428035736084
test_loss: tensor(35.9759)
epoch: 140 train_loss: 40.42439336776734
test_loss: tensor(35.9761)
epoch: 150 train_loss: 40.42055187225342
test_loss: te

epoch: 1210 train_loss: 40.42410001754761
test_loss: tensor(35.9748)
epoch: 1220 train_loss: 40.42556638717652
test_loss: tensor(35.9822)
epoch: 1230 train_loss: 40.42404956817627
test_loss: tensor(35.9777)
epoch: 1240 train_loss: 40.424192905426025
test_loss: tensor(35.9741)
epoch: 1250 train_loss: 40.425584983825686
test_loss: tensor(35.9816)
epoch: 1260 train_loss: 40.42406005859375
test_loss: tensor(35.9773)
epoch: 1270 train_loss: 40.42456741333008
test_loss: tensor(35.9741)
epoch: 1280 train_loss: 40.42039709091186
test_loss: tensor(35.9913)
epoch: 1290 train_loss: 40.42406234741211
test_loss: tensor(35.9766)
epoch: 1300 train_loss: 40.42457237243652
test_loss: tensor(35.9735)
epoch: 1310 train_loss: 40.42410650253296
test_loss: tensor(35.9743)
epoch: 1320 train_loss: 40.42411241531372
test_loss: tensor(35.9760)
epoch: 1330 train_loss: 40.424224853515625
test_loss: tensor(35.9756)
epoch: 1340 train_loss: 40.42411603927612
test_loss: tensor(35.9737)
epoch: 1350 train_loss: 40.4241

epoch: 2420 train_loss: 40.423879337310794
test_loss: tensor(35.9668)
epoch: 2430 train_loss: 40.424035263061526
test_loss: tensor(35.9741)
epoch: 2440 train_loss: 40.425429344177246
test_loss: tensor(35.9741)
epoch: 2450 train_loss: 40.42389812469482
test_loss: tensor(35.9774)
epoch: 2460 train_loss: 40.423936462402345
test_loss: tensor(35.9738)
epoch: 2470 train_loss: 40.42541351318359
test_loss: tensor(35.9811)
epoch: 2480 train_loss: 40.423900508880614
test_loss: tensor(35.9766)
epoch: 2490 train_loss: 40.42403087615967
test_loss: tensor(35.9730)
epoch: 2500 train_loss: 40.42023525238037
test_loss: tensor(35.9907)
epoch: 2510 train_loss: 40.423886966705325
test_loss: tensor(35.9759)
epoch: 2520 train_loss: 40.42444000244141
test_loss: tensor(35.9727)
epoch: 2530 train_loss: 40.423952293396
test_loss: tensor(35.9733)
epoch: 2540 train_loss: 40.42395839691162
test_loss: tensor(35.9750)
epoch: 2550 train_loss: 40.4240647315979
test_loss: tensor(35.9745)
epoch: 2560 train_loss: 40.4239

epoch: 3630 train_loss: 40.42382783889771
test_loss: tensor(35.9725)
epoch: 3640 train_loss: 40.42383737564087
test_loss: tensor(35.9743)
epoch: 3650 train_loss: 40.42347917556763
test_loss: tensor(35.9755)
epoch: 3660 train_loss: 40.42372341156006
test_loss: tensor(35.9663)
epoch: 3670 train_loss: 40.423880863189694
test_loss: tensor(35.9737)
epoch: 3680 train_loss: 40.425279235839845
test_loss: tensor(35.9737)
epoch: 3690 train_loss: 40.42374696731567
test_loss: tensor(35.9769)
epoch: 3700 train_loss: 40.42379875183106
test_loss: tensor(35.9733)
epoch: 3710 train_loss: 40.42526073455811
test_loss: tensor(35.9807)
epoch: 3720 train_loss: 40.42374439239502
test_loss: tensor(35.9762)
epoch: 3730 train_loss: 40.423886966705325
test_loss: tensor(35.9726)
epoch: 3740 train_loss: 40.42009334564209
test_loss: tensor(35.9902)
epoch: 3750 train_loss: 40.42375345230103
test_loss: tensor(35.9754)
epoch: 3760 train_loss: 40.42427005767822
test_loss: tensor(35.9723)
epoch: 3770 train_loss: 40.4238

epoch: 4840 train_loss: 40.42370443344116
test_loss: tensor(35.9668)
epoch: 4850 train_loss: 40.423733711242676
test_loss: tensor(35.9739)
epoch: 4860 train_loss: 40.423351764678955
test_loss: tensor(35.9749)
epoch: 4870 train_loss: 40.423583316802976
test_loss: tensor(35.9772)
epoch: 4880 train_loss: 40.42364559173584
test_loss: tensor(35.9734)
epoch: 4890 train_loss: 40.4251353263855
test_loss: tensor(35.9730)
epoch: 4900 train_loss: 40.4236026763916
test_loss: tensor(35.9764)
epoch: 4910 train_loss: 40.423743057250974
test_loss: tensor(35.9727)
epoch: 4920 train_loss: 40.4251145362854
test_loss: tensor(35.9800)
epoch: 4930 train_loss: 40.4236120223999
test_loss: tensor(35.9757)
epoch: 4940 train_loss: 40.42412929534912
test_loss: tensor(35.9724)
epoch: 4950 train_loss: 40.41995429992676
test_loss: tensor(35.9895)
epoch: 4960 train_loss: 40.4236026763916
test_loss: tensor(35.9749)
epoch: 4970 train_loss: 40.42378149032593
test_loss: tensor(35.9743)
epoch: 4980 train_loss: 40.42368888

In [69]:
with torch.no_grad():
    y_pred = model1(Xtest2_tensor)
y = pd.concat([pd.Series(Xtest2[:, 0].reshape(-1), name='send rates'), pd.Series(Xtest2[:, 1].reshape(-1), name='block size'), 
               pd.Series(Ytest2_tensor.numpy().reshape(-1), name='throughput_true'), 
               pd.Series(y_pred.numpy().reshape(-1), name='throughput_pred')], axis=1)
y.to_csv('./throughput_true_pred_related1.csv')

In [70]:
# Compute the test set MAE RMSE MAPE
import pandas as pd

data = pd.read_csv('./throughput_true_pred_related1.csv')
data.drop('Unnamed: 0', axis=1, inplace=True)
data.head()

,send rates,block size,throughput_true,throughput_pred
0,197.0,190.0,163.0,163.25279
1,70.2,60.0,63.2,62.05043
2,170.2,90.0,128.5,133.67148
3,170.3,200.0,159.6,144.64705
4,90.5,130.0,80.6,79.03754


In [71]:
y = data['throughput_true'].values
y

array([163. ,  63.2, 128.5, 159.6,  80.6, 129.1,  98.6, 117.4,  19.9,
       118. ,  81.3, 159.4,  83.3, 125.4, 122.6,  99.1, 134.4,  10. ,
        37.7, 119.3,  61.7, 103.5, 134.1,  96.7,  10. ,  47.8, 134.9,
        90.9, 133.5,  92.3,  50.1,  10. ,  78.3, 121.5,   9.9,  50.1,
        28.4,  53.9, 163.5,  69.8, 109.1,  92.7,  78.4,  37.3, 139.4,
        46.9, 138.7, 123.1, 121.6, 121.4,  38.3,  98.9, 103.3,  10. ,
       120.7, 116. , 121.4,  59.5, 121.9, 126.8,  55.2,  38.3,  97.2,
       103.9, 157.9,  96.5,  54.3, 120.9, 109.4, 121.8,  96.2,  48.2,
       130. , 138.2,  10. , 132.7,  30.1,  83.7,  61.6,  99.2])

In [72]:
y_hat = data['throughput_pred'].values
y_hat

array([163.25279 ,  62.05043 , 133.67148 , 144.64705 ,  79.03754 ,
       126.85877 ,  87.41215 , 127.68475 ,  21.337149, 117.75988 ,
        79.06961 , 149.71494 ,  86.81833 , 126.68283 , 119.89191 ,
       103.26853 , 147.40732 ,  12.856003,  38.27462 , 102.881065,
        61.961452, 105.49646 , 132.35321 ,  95.33157 ,  13.538788,
        46.186775, 138.94624 ,  94.51016 , 128.90276 ,  95.17842 ,
        45.479168,  12.799105,  78.55753 , 120.14911 ,  12.912899,
        45.42227 ,  30.021063,  53.78962 , 162.3078  ,  62.928726,
       119.58738 ,  94.92602 ,  78.89892 ,  37.364243, 141.42978 ,
        45.706764, 145.73715 , 127.45184 , 136.1907  , 120.31981 ,
        37.534935, 103.23645 , 103.30063 ,  13.595686, 128.05096 ,
       121.452   , 119.978424,  54.6431  , 111.90282 , 123.122696,
        54.074116,  37.648735, 103.66683 , 111.46518 , 146.63794 ,
       101.10092 ,  54.017212, 133.01202 , 119.86461 , 136.32208 ,
        95.64087 ,  46.275753, 141.53937 , 145.07794 ,  13.197

In [73]:
from sklearn import metrics

In [74]:
MAE = metrics.mean_absolute_error(y, y_hat)
RMSE = metrics.mean_squared_error(y, y_hat) ** 0.5
MAPE = metrics.mean_absolute_percentage_error(y, y_hat)
print('MAE: ', MAE)
print('RMSE: ', RMSE)
print('MAPE: ', MAPE)

MAE:  4.3392459
RMSE:  5.997928346305283
MAPE:  0.06390820799899966


# Choose the optimal block size

## MLP1 to predict latency

In [105]:
TPS = []
for i in range(10,210,10):
    for j in range(20):
        TPS.append(i)
TPS

[10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 30,
 30,
 30,
 30,
 30,
 30,
 30,
 30,
 30,
 30,
 30,
 30,
 30,
 30,
 30,
 30,
 30,
 30,
 30,
 30,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 60,
 60,
 60,
 60,
 60,
 60,
 60,
 60,
 60,
 60,
 60,
 60,
 60,
 60,
 60,
 60,
 60,
 60,
 60,
 60,
 70,
 70,
 70,
 70,
 70,
 70,
 70,
 70,
 70,
 70,
 70,
 70,
 70,
 70,
 70,
 70,
 70,
 70,
 70,
 70,
 80,
 80,
 80,
 80,
 80,
 80,
 80,
 80,
 80,
 80,
 80,
 80,
 80,
 80,
 80,
 80,
 80,
 80,
 80,
 80,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 90,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100

In [106]:
BS = list(range(10, 210, 10)) * 20
BS

[10,
 20,
 30,
 40,
 50,
 60,
 70,
 80,
 90,
 100,
 110,
 120,
 130,
 140,
 150,
 160,
 170,
 180,
 190,
 200,
 10,
 20,
 30,
 40,
 50,
 60,
 70,
 80,
 90,
 100,
 110,
 120,
 130,
 140,
 150,
 160,
 170,
 180,
 190,
 200,
 10,
 20,
 30,
 40,
 50,
 60,
 70,
 80,
 90,
 100,
 110,
 120,
 130,
 140,
 150,
 160,
 170,
 180,
 190,
 200,
 10,
 20,
 30,
 40,
 50,
 60,
 70,
 80,
 90,
 100,
 110,
 120,
 130,
 140,
 150,
 160,
 170,
 180,
 190,
 200,
 10,
 20,
 30,
 40,
 50,
 60,
 70,
 80,
 90,
 100,
 110,
 120,
 130,
 140,
 150,
 160,
 170,
 180,
 190,
 200,
 10,
 20,
 30,
 40,
 50,
 60,
 70,
 80,
 90,
 100,
 110,
 120,
 130,
 140,
 150,
 160,
 170,
 180,
 190,
 200,
 10,
 20,
 30,
 40,
 50,
 60,
 70,
 80,
 90,
 100,
 110,
 120,
 130,
 140,
 150,
 160,
 170,
 180,
 190,
 200,
 10,
 20,
 30,
 40,
 50,
 60,
 70,
 80,
 90,
 100,
 110,
 120,
 130,
 140,
 150,
 160,
 170,
 180,
 190,
 200,
 10,
 20,
 30,
 40,
 50,
 60,
 70,
 80,
 90,
 100,
 110,
 120,
 130,
 140,
 150,
 160,
 170,
 180,
 190,
 200,
 

In [107]:
len(BS)

400

In [108]:
XpBS = {"Xp": TPS, "BS": BS}
XpBS

{'Xp': [10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  20,
  20,
  20,
  20,
  20,
  20,
  20,
  20,
  20,
  20,
  20,
  20,
  20,
  20,
  20,
  20,
  20,
  20,
  20,
  20,
  30,
  30,
  30,
  30,
  30,
  30,
  30,
  30,
  30,
  30,
  30,
  30,
  30,
  30,
  30,
  30,
  30,
  30,
  30,
  30,
  40,
  40,
  40,
  40,
  40,
  40,
  40,
  40,
  40,
  40,
  40,
  40,
  40,
  40,
  40,
  40,
  40,
  40,
  40,
  40,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  60,
  60,
  60,
  60,
  60,
  60,
  60,
  60,
  60,
  60,
  60,
  60,
  60,
  60,
  60,
  60,
  60,
  60,
  60,
  60,
  70,
  70,
  70,
  70,
  70,
  70,
  70,
  70,
  70,
  70,
  70,
  70,
  70,
  70,
  70,
  70,
  70,
  70,
  70,
  70,
  80,
  80,
  80,
  80,
  80,
  80,
  80,
  80,
  80,
  80,
  80,
  80,
  80,
  80,
  80,
  80,
  80,
  80,
  80,
  80,
  90,
  90,
  90,
  90,
  90,
  90

In [109]:
XpBS = pd.DataFrame(XpBS)
XpBS

,Xp,BS
0,10,10
1,10,20
2,10,30
3,10,40
4,10,50
...,...,...
395,200,160
396,200,170
397,200,180
398,200,190


In [110]:
XpBS_values = XpBS.values
XpBS_values

array([[ 10,  10],
       [ 10,  20],
       [ 10,  30],
       [ 10,  40],
       [ 10,  50],
       [ 10,  60],
       [ 10,  70],
       [ 10,  80],
       [ 10,  90],
       [ 10, 100],
       [ 10, 110],
       [ 10, 120],
       [ 10, 130],
       [ 10, 140],
       [ 10, 150],
       [ 10, 160],
       [ 10, 170],
       [ 10, 180],
       [ 10, 190],
       [ 10, 200],
       [ 20,  10],
       [ 20,  20],
       [ 20,  30],
       [ 20,  40],
       [ 20,  50],
       [ 20,  60],
       [ 20,  70],
       [ 20,  80],
       [ 20,  90],
       [ 20, 100],
       [ 20, 110],
       [ 20, 120],
       [ 20, 130],
       [ 20, 140],
       [ 20, 150],
       [ 20, 160],
       [ 20, 170],
       [ 20, 180],
       [ 20, 190],
       [ 20, 200],
       [ 30,  10],
       [ 30,  20],
       [ 30,  30],
       [ 30,  40],
       [ 30,  50],
       [ 30,  60],
       [ 30,  70],
       [ 30,  80],
       [ 30,  90],
       [ 30, 100],
       [ 30, 110],
       [ 30, 120],
       [ 30,

In [111]:
# Normalization
XpBS_minmax = min_max_scaler1.transform(XpBS_values)
XpBS_minmax

array([[0.        , 0.        ],
       [0.        , 0.05263158],
       [0.        , 0.10526316],
       [0.        , 0.15789474],
       [0.        , 0.21052632],
       [0.        , 0.26315789],
       [0.        , 0.31578947],
       [0.        , 0.36842105],
       [0.        , 0.42105263],
       [0.        , 0.47368421],
       [0.        , 0.52631579],
       [0.        , 0.57894737],
       [0.        , 0.63157895],
       [0.        , 0.68421053],
       [0.        , 0.73684211],
       [0.        , 0.78947368],
       [0.        , 0.84210526],
       [0.        , 0.89473684],
       [0.        , 0.94736842],
       [0.        , 1.        ],
       [0.05243838, 0.        ],
       [0.05243838, 0.05263158],
       [0.05243838, 0.10526316],
       [0.05243838, 0.15789474],
       [0.05243838, 0.21052632],
       [0.05243838, 0.26315789],
       [0.05243838, 0.31578947],
       [0.05243838, 0.36842105],
       [0.05243838, 0.42105263],
       [0.05243838, 0.47368421],
       [0.

In [112]:
XpBS_tensor = torch.from_numpy(XpBS_minmax).type(torch.float32)

In [113]:
with torch.no_grad():
    latency_pred = model(XpBS_tensor)
y = pd.concat([pd.Series(XpBS_values[:, 0].reshape(-1), name='send rates'), pd.Series(XpBS_values[:, 1].reshape(-1), name='block size'), 
               pd.Series(latency_pred.numpy().reshape(-1), name='latency_pred')], axis=1)
y.index = XpBS.index
y.to_csv('./latency_pred_related1.csv')

In [114]:
y

,send rates,block size,latency_pred
0,10,10,0.313954
1,10,20,0.820229
2,10,30,1.092091
3,10,40,1.255364
4,10,50,1.405114
...,...,...,...
395,200,160,1.715595
396,200,170,1.646274
397,200,180,1.592673
398,200,190,1.570900


## MLP2 to predict throughput

In [115]:
# Normalization
XpBS_minmax2 = min_max_scaler2.transform(XpBS_values)
XpBS_minmax2

array([[0.        , 0.        ],
       [0.        , 0.05263158],
       [0.        , 0.10526316],
       [0.        , 0.15789474],
       [0.        , 0.21052632],
       [0.        , 0.26315789],
       [0.        , 0.31578947],
       [0.        , 0.36842105],
       [0.        , 0.42105263],
       [0.        , 0.47368421],
       [0.        , 0.52631579],
       [0.        , 0.57894737],
       [0.        , 0.63157895],
       [0.        , 0.68421053],
       [0.        , 0.73684211],
       [0.        , 0.78947368],
       [0.        , 0.84210526],
       [0.        , 0.89473684],
       [0.        , 0.94736842],
       [0.        , 1.        ],
       [0.05243838, 0.        ],
       [0.05243838, 0.05263158],
       [0.05243838, 0.10526316],
       [0.05243838, 0.15789474],
       [0.05243838, 0.21052632],
       [0.05243838, 0.26315789],
       [0.05243838, 0.31578947],
       [0.05243838, 0.36842105],
       [0.05243838, 0.42105263],
       [0.05243838, 0.47368421],
       [0.

In [116]:
XpBS_tensor2 = torch.from_numpy(XpBS_minmax2).type(torch.float32)

In [117]:
with torch.no_grad():
    throughput_pred = model1(XpBS_tensor2)
y = pd.concat([pd.Series(XpBS_values[:, 0].reshape(-1), name='send rates'), pd.Series(XpBS_values[:, 1].reshape(-1), name='block size'), 
               pd.Series(throughput_pred.numpy().reshape(-1), name='throughput_pred')], axis=1)
y.index = XpBS.index
y.to_csv('./throughput_pred_related1.csv')

In [118]:
y

,send rates,block size,throughput_pred
0,10,10,12.571510
1,10,20,12.628407
2,10,30,12.685304
3,10,40,12.742206
4,10,50,12.799105
...,...,...,...
395,200,160,159.231079
396,200,170,161.011200
397,200,180,162.791321
398,200,190,164.571457


In [119]:
with torch.no_grad():
    latency_pred = model(XpBS_tensor)
    throughput_pred = model1(XpBS_tensor2)
y = pd.concat([pd.Series(XpBS_values[:, 0].reshape(-1), name='send rates'), pd.Series(XpBS_values[:, 1].reshape(-1), name='block size'), 
               pd.Series(latency_pred.numpy().reshape(-1), name='latency_pred'),
               pd.Series(throughput_pred.numpy().reshape(-1), name='throughput_pred')], axis=1)
y.index = XpBS.index
y.to_csv('./latency_throughput_pred_related1.csv')

## Set the score function to chose the best block

In [120]:
y

,send rates,block size,latency_pred,throughput_pred
0,10,10,0.313954,12.571510
1,10,20,0.820229,12.628407
2,10,30,1.092091,12.685304
3,10,40,1.255364,12.742206
4,10,50,1.405114,12.799105
...,...,...,...,...
395,200,160,1.715595,159.231079
396,200,170,1.646274,161.011200
397,200,180,1.592673,162.791321
398,200,190,1.570900,164.571457


In [121]:
for name, group in y.groupby('send rates', sort=False):
    print(name)
    print(group)

10
    send rates  block size  latency_pred  throughput_pred
0           10          10      0.313954        12.571510
1           10          20      0.820229        12.628407
2           10          30      1.092091        12.685304
3           10          40      1.255364        12.742206
4           10          50      1.405114        12.799105
5           10          60      1.379416        12.856003
6           10          70      1.310039        12.912899
7           10          80      1.232086        12.969799
8           10          90      1.135677        13.026701
9           10         100      1.139989        13.083597
10          10         110      1.144301        13.140495
11          10         120      1.143092        13.197393
12          10         130      1.141496        13.254299
13          10         140      1.139900        13.311193
14          10         150      1.138304        13.368092
15          10         160      1.136708        13.424991
16         

In [122]:
la_max_min = y.groupby('send rates', sort=False)['latency_pred'].agg(['max', 'min'])
la_max_min

,max,min
send rates,,
10,1.405114,0.313954
20,1.174070,0.183035
30,1.164310,0.113699
40,1.173941,0.099700
50,1.158197,0.096375
60,1.151695,0.093051
70,1.083928,0.090240
80,0.961398,0.089850
90,0.839120,0.090206


In [123]:
la_col=['latency_max', 'latency_min']
la_max_min.columns = la_col
la_max_min

,latency_max,latency_min
send rates,,
10,1.405114,0.313954
20,1.174070,0.183035
30,1.164310,0.113699
40,1.173941,0.099700
50,1.158197,0.096375
60,1.151695,0.093051
70,1.083928,0.090240
80,0.961398,0.089850
90,0.839120,0.090206


In [124]:
th_max_min = y.groupby('send rates', sort=False)['throughput_pred'].agg(['max', 'min'])
th_max_min

,max,min
send rates,,
10,13.652584,12.571510
20,21.824415,20.743341
30,29.996248,28.915171
40,38.168083,37.087006
50,46.339912,45.258831
60,54.511738,53.430668
70,62.683575,61.602497
80,70.855408,69.774338
90,79.027245,77.946159


In [125]:
th_col=['throughput_max', 'throughput_min']
th_max_min.columns = th_col
th_max_min

,throughput_max,throughput_min
send rates,,
10,13.652584,12.571510
20,21.824415,20.743341
30,29.996248,28.915171
40,38.168083,37.087006
50,46.339912,45.258831
60,54.511738,53.430668
70,62.683575,61.602497
80,70.855408,69.774338
90,79.027245,77.946159


In [126]:
# Restore the grouping index to the column
la_max_min.reset_index(inplace=True)
la_max_min

,send rates,latency_max,latency_min
0,10,1.405114,0.313954
1,20,1.174070,0.183035
2,30,1.164310,0.113699
3,40,1.173941,0.099700
4,50,1.158197,0.096375
5,60,1.151695,0.093051
6,70,1.083928,0.090240
7,80,0.961398,0.089850
8,90,0.839120,0.090206
9,100,0.807064,0.092920


In [127]:
th_max_min.reset_index(inplace=True)
th_max_min

,send rates,throughput_max,throughput_min
0,10,13.652584,12.571510
1,20,21.824415,20.743341
2,30,29.996248,28.915171
3,40,38.168083,37.087006
4,50,46.339912,45.258831
5,60,54.511738,53.430668
6,70,62.683575,61.602497
7,80,70.855408,69.774338
8,90,79.027245,77.946159
9,100,87.199074,86.117989


In [138]:
la_max = la_max_min['latency_max'].values
la_max = list(la_max)
la_max
len(la_max)

20

In [139]:
la_max_all = []
for i in range(len(la_max)):
    la_max_all.extend([la_max[i]]*20)
    
la_max_all

[1.4051139,
 1.4051139,
 1.4051139,
 1.4051139,
 1.4051139,
 1.4051139,
 1.4051139,
 1.4051139,
 1.4051139,
 1.4051139,
 1.4051139,
 1.4051139,
 1.4051139,
 1.4051139,
 1.4051139,
 1.4051139,
 1.4051139,
 1.4051139,
 1.4051139,
 1.4051139,
 1.1740704,
 1.1740704,
 1.1740704,
 1.1740704,
 1.1740704,
 1.1740704,
 1.1740704,
 1.1740704,
 1.1740704,
 1.1740704,
 1.1740704,
 1.1740704,
 1.1740704,
 1.1740704,
 1.1740704,
 1.1740704,
 1.1740704,
 1.1740704,
 1.1740704,
 1.1740704,
 1.1643097,
 1.1643097,
 1.1643097,
 1.1643097,
 1.1643097,
 1.1643097,
 1.1643097,
 1.1643097,
 1.1643097,
 1.1643097,
 1.1643097,
 1.1643097,
 1.1643097,
 1.1643097,
 1.1643097,
 1.1643097,
 1.1643097,
 1.1643097,
 1.1643097,
 1.1643097,
 1.173941,
 1.173941,
 1.173941,
 1.173941,
 1.173941,
 1.173941,
 1.173941,
 1.173941,
 1.173941,
 1.173941,
 1.173941,
 1.173941,
 1.173941,
 1.173941,
 1.173941,
 1.173941,
 1.173941,
 1.173941,
 1.173941,
 1.173941,
 1.1581967,
 1.1581967,
 1.1581967,
 1.1581967,
 1.1581967,


In [140]:
la_min = la_max_min['latency_min'].values
la_min = list(la_min)
la_min

[0.31395432,
 0.18303476,
 0.11369938,
 0.09969959,
 0.09637543,
 0.09305138,
 0.09023982,
 0.08984975,
 0.09020576,
 0.09291993,
 0.11609901,
 0.3070148,
 0.32686746,
 0.4082949,
 0.4256592,
 0.44302332,
 0.5064016,
 0.6611626,
 0.8068545,
 0.8638486]

In [141]:
la_min_all = []
for i in range(len(la_min)):
    la_min_all.extend([la_min[i]]*20)
    
la_min_all

[0.31395432,
 0.31395432,
 0.31395432,
 0.31395432,
 0.31395432,
 0.31395432,
 0.31395432,
 0.31395432,
 0.31395432,
 0.31395432,
 0.31395432,
 0.31395432,
 0.31395432,
 0.31395432,
 0.31395432,
 0.31395432,
 0.31395432,
 0.31395432,
 0.31395432,
 0.31395432,
 0.18303476,
 0.18303476,
 0.18303476,
 0.18303476,
 0.18303476,
 0.18303476,
 0.18303476,
 0.18303476,
 0.18303476,
 0.18303476,
 0.18303476,
 0.18303476,
 0.18303476,
 0.18303476,
 0.18303476,
 0.18303476,
 0.18303476,
 0.18303476,
 0.18303476,
 0.18303476,
 0.11369938,
 0.11369938,
 0.11369938,
 0.11369938,
 0.11369938,
 0.11369938,
 0.11369938,
 0.11369938,
 0.11369938,
 0.11369938,
 0.11369938,
 0.11369938,
 0.11369938,
 0.11369938,
 0.11369938,
 0.11369938,
 0.11369938,
 0.11369938,
 0.11369938,
 0.11369938,
 0.09969959,
 0.09969959,
 0.09969959,
 0.09969959,
 0.09969959,
 0.09969959,
 0.09969959,
 0.09969959,
 0.09969959,
 0.09969959,
 0.09969959,
 0.09969959,
 0.09969959,
 0.09969959,
 0.09969959,
 0.09969959,
 0.09969959,

In [143]:
y['latency_min'] = la_min_all
y

,send rates,block size,latency_pred,throughput_pred,latency_min
0,10,10,0.313954,12.571510,0.313954
1,10,20,0.820229,12.628407,0.313954
2,10,30,1.092091,12.685304,0.313954
3,10,40,1.255364,12.742206,0.313954
4,10,50,1.405114,12.799105,0.313954
...,...,...,...,...,...
395,200,160,1.715595,159.231079,0.863849
396,200,170,1.646274,161.011200,0.863849
397,200,180,1.592673,162.791321,0.863849
398,200,190,1.570900,164.571457,0.863849


In [145]:
y['latency_max'] = la_max_all
y

,send rates,block size,latency_pred,throughput_pred,latency_min,latency_max
0,10,10,0.313954,12.571510,0.313954,1.405114
1,10,20,0.820229,12.628407,0.313954,1.405114
2,10,30,1.092091,12.685304,0.313954,1.405114
3,10,40,1.255364,12.742206,0.313954,1.405114
4,10,50,1.405114,12.799105,0.313954,1.405114
...,...,...,...,...,...,...
395,200,160,1.715595,159.231079,0.863849,2.489892
396,200,170,1.646274,161.011200,0.863849,2.489892
397,200,180,1.592673,162.791321,0.863849,2.489892
398,200,190,1.570900,164.571457,0.863849,2.489892


In [146]:
th_max = th_max_min['throughput_max'].values
th_max = list(th_max)
th_max

[13.652584,
 21.824415,
 29.996248,
 38.168083,
 46.339912,
 54.511738,
 62.683575,
 70.85541,
 79.027245,
 87.19907,
 95.370895,
 103.542725,
 111.71457,
 119.88638,
 128.05823,
 136.23006,
 144.40189,
 152.57373,
 160.74557,
 166.3516]

In [147]:
th_max_all = []
for i in range(len(th_max)):
    th_max_all.extend([th_max[i]] * 20)
    
th_max_all

[13.652584,
 13.652584,
 13.652584,
 13.652584,
 13.652584,
 13.652584,
 13.652584,
 13.652584,
 13.652584,
 13.652584,
 13.652584,
 13.652584,
 13.652584,
 13.652584,
 13.652584,
 13.652584,
 13.652584,
 13.652584,
 13.652584,
 13.652584,
 21.824415,
 21.824415,
 21.824415,
 21.824415,
 21.824415,
 21.824415,
 21.824415,
 21.824415,
 21.824415,
 21.824415,
 21.824415,
 21.824415,
 21.824415,
 21.824415,
 21.824415,
 21.824415,
 21.824415,
 21.824415,
 21.824415,
 21.824415,
 29.996248,
 29.996248,
 29.996248,
 29.996248,
 29.996248,
 29.996248,
 29.996248,
 29.996248,
 29.996248,
 29.996248,
 29.996248,
 29.996248,
 29.996248,
 29.996248,
 29.996248,
 29.996248,
 29.996248,
 29.996248,
 29.996248,
 29.996248,
 38.168083,
 38.168083,
 38.168083,
 38.168083,
 38.168083,
 38.168083,
 38.168083,
 38.168083,
 38.168083,
 38.168083,
 38.168083,
 38.168083,
 38.168083,
 38.168083,
 38.168083,
 38.168083,
 38.168083,
 38.168083,
 38.168083,
 38.168083,
 46.339912,
 46.339912,
 46.339912,
 46.

In [148]:
th_min = th_max_min['throughput_min'].values
th_min = list(th_min)
th_min

[12.57151,
 20.743341,
 28.91517,
 37.087006,
 45.25883,
 53.430668,
 61.602497,
 69.77434,
 77.94616,
 86.11799,
 92.969315,
 97.36485,
 101.76039,
 106.15594,
 110.55147,
 114.947014,
 119.342545,
 123.738075,
 128.13348,
 132.52527]

In [149]:
th_min_all = []
for i in range(len(th_min)):
    th_min_all.extend([th_min[i]] * 20)
    
th_min_all

[12.57151,
 12.57151,
 12.57151,
 12.57151,
 12.57151,
 12.57151,
 12.57151,
 12.57151,
 12.57151,
 12.57151,
 12.57151,
 12.57151,
 12.57151,
 12.57151,
 12.57151,
 12.57151,
 12.57151,
 12.57151,
 12.57151,
 12.57151,
 20.743341,
 20.743341,
 20.743341,
 20.743341,
 20.743341,
 20.743341,
 20.743341,
 20.743341,
 20.743341,
 20.743341,
 20.743341,
 20.743341,
 20.743341,
 20.743341,
 20.743341,
 20.743341,
 20.743341,
 20.743341,
 20.743341,
 20.743341,
 28.91517,
 28.91517,
 28.91517,
 28.91517,
 28.91517,
 28.91517,
 28.91517,
 28.91517,
 28.91517,
 28.91517,
 28.91517,
 28.91517,
 28.91517,
 28.91517,
 28.91517,
 28.91517,
 28.91517,
 28.91517,
 28.91517,
 28.91517,
 37.087006,
 37.087006,
 37.087006,
 37.087006,
 37.087006,
 37.087006,
 37.087006,
 37.087006,
 37.087006,
 37.087006,
 37.087006,
 37.087006,
 37.087006,
 37.087006,
 37.087006,
 37.087006,
 37.087006,
 37.087006,
 37.087006,
 37.087006,
 45.25883,
 45.25883,
 45.25883,
 45.25883,
 45.25883,
 45.25883,
 45.25883,
 45

In [150]:
y['throughput_min'] = th_min_all
y

,send rates,block size,latency_pred,throughput_pred,latency_min,latency_max,throughput_min
0,10,10,0.313954,12.571510,0.313954,1.405114,12.571510
1,10,20,0.820229,12.628407,0.313954,1.405114,12.571510
2,10,30,1.092091,12.685304,0.313954,1.405114,12.571510
3,10,40,1.255364,12.742206,0.313954,1.405114,12.571510
4,10,50,1.405114,12.799105,0.313954,1.405114,12.571510
...,...,...,...,...,...,...,...
395,200,160,1.715595,159.231079,0.863849,2.489892,132.525269
396,200,170,1.646274,161.011200,0.863849,2.489892,132.525269
397,200,180,1.592673,162.791321,0.863849,2.489892,132.525269
398,200,190,1.570900,164.571457,0.863849,2.489892,132.525269


In [151]:
y['throughput_max'] = th_max_all
y

,send rates,block size,latency_pred,throughput_pred,latency_min,latency_max,throughput_min,throughput_max
0,10,10,0.313954,12.571510,0.313954,1.405114,12.571510,13.652584
1,10,20,0.820229,12.628407,0.313954,1.405114,12.571510,13.652584
2,10,30,1.092091,12.685304,0.313954,1.405114,12.571510,13.652584
3,10,40,1.255364,12.742206,0.313954,1.405114,12.571510,13.652584
4,10,50,1.405114,12.799105,0.313954,1.405114,12.571510,13.652584
...,...,...,...,...,...,...,...,...
395,200,160,1.715595,159.231079,0.863849,2.489892,132.525269,166.351593
396,200,170,1.646274,161.011200,0.863849,2.489892,132.525269,166.351593
397,200,180,1.592673,162.791321,0.863849,2.489892,132.525269,166.351593
398,200,190,1.570900,164.571457,0.863849,2.489892,132.525269,166.351593


In [152]:
y.to_csv('./latency_throughput_pred_max_min_related1.csv')

### choose different weights

In [154]:
# Read the processed data directly
y = pd.read_csv('./latency_throughput_pred_max_min_related1.csv')
y

,Unnamed: 0,send rates,block size,latency_pred,throughput_pred,latency_min,latency_max,throughput_min,throughput_max
0,0,10,10,0.313954,12.571510,0.313954,1.405114,12.57151,13.652584
1,1,10,20,0.820229,12.628407,0.313954,1.405114,12.57151,13.652584
2,2,10,30,1.092091,12.685304,0.313954,1.405114,12.57151,13.652584
3,3,10,40,1.255364,12.742206,0.313954,1.405114,12.57151,13.652584
4,4,10,50,1.405114,12.799105,0.313954,1.405114,12.57151,13.652584
...,...,...,...,...,...,...,...,...,...
395,395,200,160,1.715595,159.231080,0.863849,2.489892,132.52527,166.351600
396,396,200,170,1.646274,161.011200,0.863849,2.489892,132.52527,166.351600
397,397,200,180,1.592673,162.791320,0.863849,2.489892,132.52527,166.351600
398,398,200,190,1.570900,164.571460,0.863849,2.489892,132.52527,166.351600


In [187]:
alpha = 0.1
beta = 0.9

In [188]:
latency_score = alpha * ((y['latency_max'] - y['latency_pred']) / (y['latency_max'] - y['latency_min']))

In [189]:
throughput_score = beta * ((y['throughput_pred'] - y['throughput_min']) / (y['throughput_max'] - y['throughput_min']))

In [190]:
y_score = latency_score.values + throughput_score.values
y_score

array([0.1       , 0.10096964, 0.12342132, 0.15582929, 0.18947408,
       0.23919705, 0.29292144, 0.34743498, 0.40364173, 0.45061277,
       0.49758544, 0.54506418, 0.59258503, 0.64009582, 0.68761084,
       0.73512583, 0.78264166, 0.83013237, 0.87696139, 0.92374973,
       0.1       , 0.11329457, 0.13400128, 0.15911894, 0.19093054,
       0.24089647, 0.28499624, 0.33158109, 0.38246714, 0.4294049 ,
       0.47633602, 0.52371698, 0.57125094, 0.61877901, 0.66630296,
       0.71383691, 0.76136499, 0.80887057, 0.85564514, 0.90195675,
       0.1       , 0.1233097 , 0.15533286, 0.1851285 , 0.21640996,
       0.25584254, 0.29240697, 0.33868652, 0.38248324, 0.42796894,
       0.47492621, 0.52220454, 0.56972169, 0.61724132, 0.66476093,
       0.71228559, 0.75980354, 0.80730096, 0.85410733, 0.9       ,
       0.1       , 0.13194943, 0.16609086, 0.20268778, 0.23333539,
       0.27042973, 0.3072434 , 0.35046334, 0.38715533, 0.43055113,
       0.47547211, 0.52264468, 0.57016267, 0.61767818, 0.66519

In [191]:
y['score'] = y_score

In [192]:
y.to_csv('./latency_throughput_score_alpha' + str(alpha) + '_beta' + str(beta) + '_related1.csv')

In [193]:
y

,Unnamed: 0,send rates,block size,latency_pred,throughput_pred,latency_min,latency_max,throughput_min,throughput_max,score
0,0,10,10,0.313954,12.571510,0.313954,1.405114,12.57151,13.652584,0.100000
1,1,10,20,0.820229,12.628407,0.313954,1.405114,12.57151,13.652584,0.100970
2,2,10,30,1.092091,12.685304,0.313954,1.405114,12.57151,13.652584,0.123421
3,3,10,40,1.255364,12.742206,0.313954,1.405114,12.57151,13.652584,0.155829
4,4,10,50,1.405114,12.799105,0.313954,1.405114,12.57151,13.652584,0.189474
...,...,...,...,...,...,...,...,...,...,...
395,395,200,160,1.715595,159.231080,0.863849,2.489892,132.52527,166.351600,0.758166
396,396,200,170,1.646274,161.011200,0.863849,2.489892,132.52527,166.351600,0.809792
397,397,200,180,1.592673,162.791320,0.863849,2.489892,132.52527,166.351600,0.860451
398,398,200,190,1.570900,164.571460,0.863849,2.489892,132.52527,166.351600,0.909154


In [194]:
# for name, group in y_filter.groupby('send rates', sort=False):
#     print(name)
#     print(group)

In [185]:
score_max = y_filter.groupby('send rates', sort=False)['score'].agg(['max'])
score_max

,max
send rates,
110,0.900000
120,0.957629
130,0.935767
140,0.975530
150,0.959464
160,0.948897
170,0.942622
180,0.937040
190,0.932751


In [186]:
score_col=['score_max']
score_max.columns = score_col
score_max

,score_max
send rates,
110,0.900000
120,0.957629
130,0.935767
140,0.975530
150,0.959464
160,0.948897
170,0.942622
180,0.937040
190,0.932751


In [173]:
score_max.reset_index(inplace=True)
score_max

,send rates,score_max
0,110,0.900000
1,120,0.957629
2,130,0.935767
3,140,0.975530
4,150,0.959464
5,160,0.948897
6,170,0.942622
7,180,0.937040
8,190,0.932751
9,200,0.931587


In [174]:
score_max = score_max['score_max'].values
score_max = list(score_max)
score_max

[0.9,
 0.9576292009793012,
 0.935766542296804,
 0.9755297281665295,
 0.9594641954417422,
 0.948897047652697,
 0.9426218442660812,
 0.9370401678061414,
 0.9327513508027239,
 0.9315867846142339]

In [175]:
score_max_all = []
for i in range(len(score_max)):
    score_max_all.extend([score_max[i]] * 20)
    
score_max_all

[0.9,
 0.9,
 0.9,
 0.9,
 0.9,
 0.9,
 0.9,
 0.9,
 0.9,
 0.9,
 0.9,
 0.9,
 0.9,
 0.9,
 0.9,
 0.9,
 0.9,
 0.9,
 0.9,
 0.9,
 0.9576292009793012,
 0.9576292009793012,
 0.9576292009793012,
 0.9576292009793012,
 0.9576292009793012,
 0.9576292009793012,
 0.9576292009793012,
 0.9576292009793012,
 0.9576292009793012,
 0.9576292009793012,
 0.9576292009793012,
 0.9576292009793012,
 0.9576292009793012,
 0.9576292009793012,
 0.9576292009793012,
 0.9576292009793012,
 0.9576292009793012,
 0.9576292009793012,
 0.9576292009793012,
 0.9576292009793012,
 0.935766542296804,
 0.935766542296804,
 0.935766542296804,
 0.935766542296804,
 0.935766542296804,
 0.935766542296804,
 0.935766542296804,
 0.935766542296804,
 0.935766542296804,
 0.935766542296804,
 0.935766542296804,
 0.935766542296804,
 0.935766542296804,
 0.935766542296804,
 0.935766542296804,
 0.935766542296804,
 0.935766542296804,
 0.935766542296804,
 0.935766542296804,
 0.935766542296804,
 0.9755297281665295,
 0.9755297281665295,
 0.975529728166529

In [177]:
y_filter['score_max'] = score_max_all
y_filter

C:\Users\cswan\AppData\Local\Temp\ipykernel_1808\1654378504.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_filter['score_max'] = score_max_all


,Unnamed: 0,send rates,block size,latency_pred,throughput_pred,latency_min,latency_max,throughput_min,throughput_max,score,score_max
200,200,110,10,0.116099,92.969315,0.116099,0.824487,92.969315,95.370895,0.900000,0.900000
201,201,110,20,0.451027,94.346720,0.116099,0.824487,92.969315,95.370895,0.531831,0.900000
202,202,110,30,0.262916,94.403610,0.116099,0.824487,92.969315,95.370895,0.773194,0.900000
203,203,110,40,0.513995,94.460520,0.116099,0.824487,92.969315,95.370895,0.456569,0.900000
204,204,110,50,0.426948,94.517420,0.116099,0.824487,92.969315,95.370895,0.569531,0.900000
...,...,...,...,...,...,...,...,...,...,...,...
395,395,200,160,1.715595,159.231080,0.863849,2.489892,132.525270,166.351600,0.507516,0.931587
396,396,200,170,1.646274,161.011200,0.863849,2.489892,132.525270,166.351600,0.551147,0.931587
397,397,200,180,1.592673,162.791320,0.863849,2.489892,132.525270,166.351600,0.586077,0.931587
398,398,200,190,1.570900,164.571460,0.863849,2.489892,132.525270,166.351600,0.603391,0.931587


In [178]:
result = y_filter[y_filter['score'] == y_filter['score_max']]
result

,Unnamed: 0,send rates,block size,latency_pred,throughput_pred,latency_min,latency_max,throughput_min,throughput_max,score,score_max
200,200,110,10,0.116099,92.969315,0.116099,0.824487,92.969315,95.370895,0.900000,0.900000
222,222,120,30,0.307015,100.925110,0.307015,0.866201,97.364850,103.542725,0.957629,0.957629
242,242,130,30,0.326867,105.320656,0.326867,0.896848,101.760390,111.714570,0.935767,0.935767
266,266,140,70,0.410121,116.836700,0.408295,1.135574,106.155940,119.886380,0.975530,0.975530
286,286,150,70,0.427485,121.232240,0.425659,1.488943,110.551470,128.058230,0.959464,0.959464
306,306,160,70,0.444849,125.627770,0.443023,1.719399,114.947014,136.230060,0.948897,0.948897
326,326,170,70,0.506402,130.023300,0.506402,1.949942,119.342545,144.401890,0.942622,0.942622
346,346,180,70,0.661163,134.418850,0.661163,2.181794,123.738075,152.573730,0.937040,0.937040
366,366,190,70,0.806855,138.814380,0.806855,2.339685,128.133480,160.745570,0.932751,0.932751
386,386,200,70,0.863849,143.209920,0.863849,2.489892,132.525270,166.351600,0.931587,0.931587


In [179]:
result.to_csv('./best_block_size_alpha' + str(alpha) + '_beta' + str(beta) + '_related1.csv')